Symbolic verification of the matrix reduction computing the basis of the union of ideals $J_1$ and $J_2$.

We use the gcd column operation introduced in the previous Lemma:

In [1]:
def gcd_operation(M, idx1, idx2, g, u, v, c1i, c2i):
    """
    Given matrix M performs gcd column operations.
    'idx1' - The index of the first column.
    'idx2' - The index of the second column.
    'g', 'u', 'v', 'c1i', 'c2i' - Gives the (potentially scaled gcd equation to use)
        g = u*cli + v*c2i
    """
    columns = M.columns()
    col1 = columns[idx1]
    col2 = columns[idx2]
    newcol1 = u * col1 + v * col2
    newcol2 = -(c2i/g)*col1 + (c1i/g)*col2
    columns[idx1] = newcol1
    columns[idx2] = newcol2
    return column_matrix(columns)

Now we do do the reduction symbolically:

In [2]:
RR.<A, B, p, ln, lg, lh, N, alpha, beta, Aprime, Bprime, gamma, delta> = PolynomialRing(Rationals())

# The matrix we start with
union_basis = matrix([
    [ 1,           0,  (p*alpha*ln)/N,    (p*beta*ln)/N,  1,   0,    alpha/ln,      beta/ln ],
    [ 0,  1/(2*N) -1,   (p*beta*ln)/N,  -(p*alpha*ln)/N,  0,  -1,    -beta/ln,     alpha/ln ],
    [ 0,     -beta/N,        ln/(2*N),                0,  0,   0, -1/(2*ln*p),            0 ],
    [ 0,     alpha/N,               0,         ln/(2*N),  0,   0,            0, -1/(2*p*ln) ]])

# Apply column operations from proof
# Warning: here columns are indexed from 0, in the paper they are indexed from 1

union_basis.add_multiple_of_column(1, 5, -1)  # c_2 -> c_2 - c_6
union_basis = union_basis * diagonal_matrix([1,1,1,1,1,-1,1,1]) # change sign of col 6
union_basis.add_multiple_of_column(4, 0, -1)
union_basis = gcd_operation(union_basis, 2, 6, lg, A, -B, p*ln, -N/ln)
union_basis = gcd_operation(union_basis, 3, 7, lg, A, -B, p*ln, -N/ln)
union_basis = gcd_operation(union_basis, 0, 6, lh, -Bprime, Aprime, lg, 2*alpha*p)
union_basis = gcd_operation(union_basis, 0, 7, 1, gamma, -delta, lh, 2*beta*p)
union_basis = union_basis * diagonal_matrix([1,1,1,-1,1,1,1,1]) # change sign of col 4
union_basis = gcd_operation(union_basis, 1, 3, lh, Aprime, Bprime, 2*alpha*p, -lg)
union_basis = gcd_operation(union_basis, 2, 3, 1, gamma, delta, lh, -2*beta*p)
union_basis.add_multiple_of_column(3, 5, -2*lg)
union_basis.add_multiple_of_column(6, 4, -(2*alpha*p)/lh)
union_basis.add_multiple_of_column(7, 4, 2*p*beta*Bprime)
union_basis = union_basis * diagonal_matrix([1,1,1,1,-1,1,1,-1]) # change sign of col 5
union_basis.swap_columns(1, 3)
union_basis.swap_columns(1, 5)

# Now we substitute  gamma = (1 + 2*delta*beta*p)/lh  from the gcd relation
union_basis = matrix(4, 8, [r(gamma=(1 + 2*delta*beta*p)/lh) for r in union_basis.list()])

# And  Bprime = (Aprime*2*alpha*p - lh)/lg
union_basis = matrix(4, 8, [r(Bprime = (Aprime*2*alpha*p - lh)/lg) for r in union_basis.list()])

# And  B = (lg - A*p*ln)*ln/N
union_basis = matrix(4, 8, [r(B = (lg - A*p*ln)*ln/N) for r in union_basis.list()])

In [3]:
union_basis

[                                                                                             1/lg                                                                                                 0                                                                (2*A*p*ln*alpha - lg*alpha)/(lh*N) (-4*A*p^2*ln*alpha*beta*Aprime + 2*A*p*ln*lh*beta + 2*p*lg*alpha*beta*Aprime - lg*lh*beta)/(lg*N)                                                                                                 0                                                                                                 0                                                                                                 0                                                                                                 0]
[                                                                                                0                                                                                                 1                  

Almost all these entries are as stated in the paper - with 4 exceptions which we check below.

In [4]:
union_basis.column(2)[1]

(4*p*lg*alpha^2*delta + 4*p*lg*beta^2*delta + 2*lg*beta + lg*delta)/(2*lh*N)

That is:
$$\frac{l^{g-h} \delta (4p \alpha^2 + 4p\beta^2) + 2l^{g-h} \beta + l^{g-h} \delta}{2N}$$
For which we use the definition of $N$.
$$4N = 1 + 4p\alpha^2 + 4p\beta^2$$
so the above value becomes
$$\frac{l^{g-h} \delta (4N-1) + 2l^{g-h} \beta + l^{g-h} \delta}{2N}$$
Giving, as it is in the paper,
$$\frac{l^{g-h} (\beta + 2N\delta)}{N}$$


In [5]:
union_basis.column(3)[0]

(-4*A*p^2*ln*alpha*beta*Aprime + 2*A*p*ln*lh*beta + 2*p*lg*alpha*beta*Aprime - lg*lh*beta)/(lg*N)

$$\frac{-4Ap^2 l^{n-g} \alpha \beta A' + 2Apl^{n-g}l^h\beta + 2p\alpha \beta A' - l^h \beta}{N}$$
Where we substitute $2\alpha p A' = l^g B' + l^h$
$$\frac{-2Ap l^{n-g} \beta (l^g B' + l^h) + 2Apl^{n-g}l^h\beta + \beta (l^g B' + l^h) - l^h \beta}{N}$$
Which simplifies to the result in the paper
$$\frac{-2Ap l^n \beta B' + \beta l^g B'}{N}$$

In [6]:
union_basis.column(3)[1]

(4*p*alpha^2*Aprime - 2*lh*alpha + Aprime)/(2*N)

$$\frac{4p\alpha^2 A' - 2l^h \alpha + A'}{2N}$$
Again we use the relation $2\alpha p A' = l^g B' + l^h$
$$\frac{2\alpha (l^g B' + l^h) - 2l^h \alpha + A'}{2N}$$
Which simplifies to the correct value
$$\frac{2 B' l^g \alpha + A'}{2N}$$


In [7]:
union_basis.column(5)[1]

(4*p*lg*alpha^2 + 4*p*lg*beta^2 - 4*lg*N + lg)/(2*N)

$$\frac{4pl^g \alpha^2 + 4pl^g \beta^2 - 4l^g N + l^g}{2N}$$

We use $4N = 1 + 4p(\alpha^2 + \beta^2)$

$$\frac{4p l^g(\alpha^2 + \beta^2) - 4l^g N + l^g}{2N} = \frac{l^g(4N - 1) - 4l^g N + l^g}{2N} = 0$$